In [ ]:
import psycopg2

In [ ]:
conString = {'user' = "postgres", 'password' = "techx007", 'host' = "localhost", 'port' = "5432", 'database' = "mydb"}

In [ ]:
def getDBConnection(conString):
    try:
        connection = psycopg2.connect(user = conString[0], 
                                      password = conString[1],
                                      host = conString[2], 
                                      port = conString[3], 
                                      database = conString[4])
        
        connection.set_session(autocommit=True)
        print('Connection established successfully.')
    except(Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error):
            
    return connection

def closeConnection(conn):
    #closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")

In [ ]:
def createEmployeeTable(conString):
    command = '''
                CREATE TABLE Employee(
                    EmployeeID serial PRIMARY KEY,
                    EmployeeName varchar(50) NOT NULL,
                    ManagerID int,
                    FOREIGN KEY (ManagerID) REFERENCES Employee(EmployeeID)
                    ON DELETE SET NULL
                );'''
    conn = getDBConnection(conString)
    try:
        cursor = conn.cursor()
        
        cursor.execute(command)
        print('Table Successfully Created.')
        
    except(Exception, psycopg2.Error) as error :
        print ("Error while creating table.", error)
    finally:
        cursor.close()
        closeConnection(conn)
    

In [ ]:
def insertEmployeeData(conString:dict, records:list):
    command = 'INSERT INTO Employee (EmployeeName, ManagerID) VALUES (%s, %s, %s)'
    conn = getDBConnection(conString)
    try:
        cursor = conn.cursor()
        
        cursor.executemany(command, records)
        print(cursor.rowcount, 'records inserted successfully.')
        
    except(Exception, psycopg2.Error) as error :
        print ("Error while inserting data.", error)
    
     finally:
        cursor.close()
        closeConnection(conn)   
# records = [('John', NULL),('Mark', NULL),('Steve', NULL),('Tom', NULL),('Lara', NULL),('Simon', NULL),('David', NULL),('Ben', NULL),('Stacy', NULL),('Sam', NULL)]
        

In [ ]:
def retrieveEmployees(conString:dict, employeeIDs:list = []):
    conn = getDBConnection(conString)
    
    if employeeIDs:
        command = 'SELECT * FROM Employee;'
        try:
            cursor = conn.cursor()
            cursor.execute(command)
            
            employees_ = cursor.fetchall()
            
            return employees_
        
        except(Exception, psycopg2.Error) as error:
            print ("Unable to retrieve data. ", error)
        finally:
        cursor.close()
        closeConnection(conn)
    else:
        command = 'SELECT * FROM Employee WHERE EmployeeID in (%s);'
        try:
            cursor = conn.cursor()
            cursor.executemany(command, employeeIDs)
            
            employees_ = cursor.fetchall()
            
            return employees_
        
        except(Exception, psycopg2.Error) as error:
            print ("Unable to retrieve data. ", error)
        finally:
        cursor.close()
        closeConnection(conn)
    

In [ ]:
def deleteEmployees(conString:dict, employeeIDs:list = []):
    conn = getDBConnection(conString)
    
    if employeeIDs:
        command = 'DELETE FROM Employee;'
        try:
            cursor = conn.cursor()
            cursor.execute(command)
            
            print(cursor.rowcount, 'records removed.')
            
        except(Exception, psycopg2.Error) as error:
            print ("Unable to delete data. ", error)
        finally:
        cursor.close()
        closeConnection(conn)
    else:
        command = 'DELETE FROM Employee WHERE EmployeeID in (%s);'
        try:
            cursor = conn.cursor()
            cursor.executemany(command, employeeIDs)
            
            print(cursor.rowcount, 'records removed.')
        
        except(Exception, psycopg2.Error) as error:
            print ("Unable to delete data. ", error)
        finally:
        cursor.close()
        closeConnection(conn)
    

In [ ]:
def updateOrgHierarchy(conString:dict, orgHierarchy:list = []):
    conn = getDBConnection(conString)
    
    command = 'UPDATE Employee SET ManagerID = %s WHERE EmployeeID = %s;'
    try:
        cursor = conn.cursor()
        cursor.executemany(command, orgHierarchy)

        print(cursor.rowcount, 'records updated.')

    except(Exception, psycopg2.Error) as error:
        print ("Unable to update data. ", error)
    finally:
    cursor.close()
    closeConnection(conn)
# orgHierarchy = [(12, 19), (13, 12), (14,17), (15,12), (16,14), (17, 19), (18, 15), (20, 17), (21, 14)] # (emp, manager)
    